<a href="https://colab.research.google.com/github/python-freak/Sentiment-Analysis/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/python-freak/Sentiment-Analysis

Cloning into 'Sentiment-Analysis'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 2), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [0]:
! pwd
%cd ./Sentiment-Analysis

/content
/content/Sentiment-Analysis


In [0]:
import imdb

In [0]:
imdb.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [0]:
x_train_text , y_train = imdb.load_data(train = True)
x_test_text  , y_test  = imdb.load_data(train = False)

In [0]:
print(len(x_train_text))
print(len(x_test_text))

25000
25000


In [0]:
x_train_text[2499]

"I admit I had some trepidation when I first saw the previews for this film. Was VH-1 treading on hollow ground here? I mean, Harris and Quinn don't really look or even sound like John or Paul. But I have to admit, this film really surprised me. It's far from the exploitation film I expected. Instead, it's a character study, a low-key, whimsical, and ultimately bittersweet look at friendship, and the ultimate lesson we all learn: it's hard, if not impossible, to capture what we once had, and what has passed us by."

In [0]:
from keras.preprocessing import text
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding , GRU , Dense
import numpy as np

Using TensorFlow backend.


In [0]:
data = x_train_text + x_test_text
num_words = 20000

In [0]:
tokenizer = text.Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)
word_ids = tokenizer.word_index
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [0]:
type(x_train_tokens[1])
#np.array(x_train_tokens[1])
np.array(x_test_tokens[1])

array([  170,   299,  2519,   662,  1498,    95,   334,   159,     1,
        5973,     4,    11,    17,    11,    17,     6,     1,   289,
         471,     4,    85,     3,  2005,   182,    17,   142,    26,
           7,     7,     1,   111,   184,     5,    26,     3,   226,
        1456,    30,    86,   805,    18,     9,    80,   406,  1256,
           3,   226,    42,   772,   166,    20,     2,    22,    62,
         178,     5,   635,     9,    15,     1,   336,    55,    18,
         100,   336,   805,    22,    80,   375,     5,  2824,     1,
        1278,   294,     2,    91,    22,    80,   118,    85,   491,
           3,    17,    67,    26,     7,     7,     3,   662,     4,
        1952,    15,  1254,   493,   455,   776,   243,    36,    11,
          17,    11,     6,    21,   125,   961,  1908,  1660,  1254,
          17,     8,    60,    22,    39,  4391,   122,   125,  1128,
           2,   846,     8,   982,     4,     1,   258,    39,     5,
          63,   191,

In [0]:
# creating inverse dictionary
id2word = {id:word for word,id in word_ids.items()}
#id2word

In [0]:
# generate text from tokens
def tokens_to_text(tokens):
  words = []
  for token in tokens:
    words.append(id2word[token])
  text = ' '.join(words)
  return text
  

In [0]:
text = tokens_to_text(x_train_tokens[1])
print(x_train_text[1])
#print(x_train_tokens[1])
print('\n')
print(text)

This is one of my all-time favorite films, and while it may move too slowly for some, it's well worth seeing. A corporate lawyer (Richard Chamberlain) is dragged into a case involving "city" Aborigines, and this is no ordinary case. OK, a man has died but it wasn't exactly a normal killing. There has also been a greater than average amount of rain lately, and the atmosphere of most of the film is somewhat claustrophobic & oppressive. The Aborigines are harboring a secret and refuse to spill the beans. This has a lot to do with white men making assumptions about "City" vs. "Tribal" Aborigines, and of course no Abo in the big city would practice tribal ways. Uh huh. Chamberlain is having strange dreams and he is somehow the key to what's happening, although no matter how many times I've seen this I can't quite grasp the exact connection. This is a very eerie and creepy film, and is a fine example of Peter Weir's ability to create tension out of nothing. The ending is a little ambiguous b

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
print(num_tokens)
num_tokens = np.array(num_tokens)
max_num_tokens = np.max(num_tokens)
print(max_num_tokens)

[115, 196, 184, 270, 79, 66, 64, 175, 366, 145, 353, 423, 87, 113, 83, 61, 471, 265, 63, 429, 186, 220, 119, 88, 166, 185, 56, 135, 300, 298, 630, 416, 201, 229, 238, 276, 590, 171, 483, 108, 124, 675, 352, 452, 194, 109, 157, 151, 79, 94, 109, 151, 235, 155, 439, 103, 151, 254, 178, 385, 173, 131, 787, 561, 421, 459, 329, 143, 154, 123, 174, 97, 88, 294, 201, 267, 153, 296, 342, 89, 294, 283, 153, 161, 67, 275, 130, 599, 133, 354, 152, 112, 89, 537, 129, 78, 136, 109, 131, 176, 264, 261, 184, 295, 121, 160, 582, 115, 177, 93, 117, 169, 100, 93, 465, 175, 95, 119, 127, 168, 224, 198, 129, 503, 110, 268, 53, 253, 570, 160, 235, 185, 125, 70, 140, 39, 125, 267, 249, 220, 152, 106, 56, 194, 121, 185, 188, 137, 102, 349, 51, 1277, 120, 198, 984, 584, 112, 214, 123, 77, 245, 367, 722, 164, 816, 182, 84, 301, 195, 254, 138, 116, 264, 330, 125, 141, 217, 151, 121, 294, 147, 479, 425, 356, 114, 766, 300, 130, 141, 73, 300, 234, 242, 331, 711, 114, 195, 482, 283, 136, 113, 169, 172, 114, 174, 1

In [0]:
mean = np.mean(num_tokens)
std =  np.std(num_tokens)


In [0]:
max_sequence_length = int(mean + 2 * std)
print(max_sequence_length)
np.sum(num_tokens < max_sequence_length) / len(num_tokens)



563


0.94546

In [0]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens , maxlen = max_sequence_length , padding = pad , truncating = pad)
#x_train_pad[1]
x_test_pad = pad_sequences(x_test_tokens , maxlen = max_sequence_length , padding = pad , truncating = pad)

In [0]:
# model
embedding_size = 128 
optimizer = Adam(lr = 1e-3)
model = Sequential()
model.add(Embedding(input_dim = num_words+1 , output_dim = embedding_size , embeddings_initializer = 'uniform' , input_length = max_sequence_length , name = 'Embedding_layer' ))
model.add(GRU(units = 64 , return_sequences = True))
#model.add(GRU(units = 64  , return_sequences = True))
model.add(GRU(units = 32))
model.add(Dense(4 , activation = 'relu'))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(optimizer = optimizer , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [0]:
model.fit(x_train_pad , y_train , batch_size = 64 , epochs = 15 , validation_split = 0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/15
20000/20000 [==============================] - 662s 33ms/step - loss: 0.4534 - acc: 0.7804 - val_loss: 0.6235 - val_acc: 0.7472
Epoch 2/15
20000/20000 [==============================] - 660s 33ms/step - loss: 0.2212 - acc: 0.9163 - val_loss: 0.4617 - val_acc: 0.7966
Epoch 3/15
20000/20000 [==============================] - 661s 33ms/step - loss: 0.1343 - acc: 0.9546 - val_loss: 0.2148 - val_acc: 0.9174
Epoch 4/15
20000/20000 [==============================] - 661s 33ms/step - loss: 0.0889 - acc: 0.9696 - val_loss: 0.4636 - val_acc: 0.8408
Epoch 5/15
20000/20000 [==============================] - 660s 33ms/step - loss: 0.1461 - acc: 0.9407 - val_loss: 0.9696 - val_acc: 0.6578
Epoch 6/15
20000/20000 [==============================] - 663s 33ms/step - loss: 0.0774 - acc: 0.9727 - val_loss: 0.5116 - val_acc: 0.8564
Epoch 7/15
20000/20000 [==============================] - 659s 33ms/step - loss: 0.0273 - acc: 0.9919 - val_loss: 0.8

In [0]:
model.predict(x_test_pad)

array([[9.9970055e-01],
       [9.9991262e-01],
       [1.5115509e-03],
       ...,
       [4.2207027e-04],
       [7.7595061e-05],
       [9.9902415e-01]], dtype=float32)

In [0]:
weights = model.get_weights()

In [0]:
! pip install h5py

In [0]:
from keras.models import model_from_json

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [0]:
test_predictions = model.predict(x_test_pad)

In [0]:
test_predictions[test_predictions > 0.5] = 1
test_predictions[test_predictions != 1] = 0
test_predictions.shape

(25000, 1)

In [0]:
actual_predictions = y_test
type(actual_predictions)
actual_predictions = np.array(actual_predictions)
actial_predictions = np.reshape(actual_predictions , (25000,1))
actual_predictions.shape


(25000,)

In [0]:
test_accuracy = float(np.sum(test_predictions == actual_predictions) )/ len(test_predictions)
test_accuracy

12500.0

In [0]:
_  , test_acc = model.evaluate(x_test_pad , y_test , batch_size = 64)
test_acc

25000/25000 [==============================] - 298s 12ms/step


0.83868

In [0]:
def preprocess_text(text):
  input = []
  input.append(text)
  tokens = tokenizer.texts_to_sequences(input)
  #tokens = np.array(tokens)
  tokens_pad = pad_sequences(tokens, maxlen = max_sequence_length ,  padding = 'pre' , truncating = 'pre')
  #input.append(tokens_pad)
  return tokens_pad

In [0]:
text = "the movie was fantastic"
#tokens = tokenizer.texts_to_sequences(text)
#print(tokens)
tokens_pad = preprocess_text(text)
#len(tokens_pad)
y = model.predict(tokens_pad)[0][0]
#y = y[0][0]
print(y)

0.9702021
